In [56]:
import pandas as pd
import numpy as np
from pymongo import MongoClient
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import mean_squared_error, r2_score
import joblib

In [25]:
#Createing the connection to the database part 1
mongo = MongoClient(port=27017)
db = mongo['hockey_project_db']
db.list_collection_names()

['2022-2023', '2023-2024']

In [26]:
#creating connections part 2: putting it into a dataframe
data_2022_2023 = db['2022-2023']
answer = list(data_2022_2023.find({}))
df_2022 = pd.json_normalize(answer)

In [ ]:
df_2022.head()

In [4]:
corr = df_2022.corr()
corr.style.background_gradient(cmap='coolwarm')

c:\Users\evan-\anaconda3\envs\PythonData\lib\site-packages\pandas\io\formats\style.py:1089: RuntimeWarning: All-NaN slice encountered
  smin = np.nanmin(s.to_numpy()) if vmin is None else vmin
c:\Users\evan-\anaconda3\envs\PythonData\lib\site-packages\pandas\io\formats\style.py:1090: RuntimeWarning: All-NaN slice encountered
  smax = np.nanmax(s.to_numpy()) if vmax is None else vmax


,,playerId,season,games_played,minutes/game,I_F_xOnGoal,I_F_xGoals,I_F_xRebounds,I_F_primaryAssists,I_F_secondaryAssists,I_F_shotAttempts,I_F_points,I_F_goals,I_F_rebounds,I_F_reboundGoals,penalties,I_F_penalityMinutes,I_F_hits,I_F_takeaways,I_F_giveaways,I_F_lowDangerShots,I_F_mediumDangerShots,I_F_highDangerShots,I_F_lowDangerxGoals,I_F_mediumDangerxGoals,I_F_highDangerxGoals,I_F_lowDangerGoals,I_F_mediumDangerGoals,I_F_highDangerGoals,I_F_unblockedShotAttempts
,1.000000,0.065788,nan,-0.061706,-0.015724,0.032409,0.049689,0.042071,0.048165,0.049761,0.021270,0.057026,0.056553,0.021392,0.013044,-0.077233,-0.064519,-0.058391,-0.003229,-0.015948,0.014117,0.041725,0.053911,0.036871,0.040859,0.056830,0.028575,0.076378,0.041659,0.028754
playerId,0.065788,1.000000,nan,-0.277929,-0.186240,-0.216005,-0.168990,-0.200261,-0.165493,-0.181701,-0.225504,-0.172020,-0.132537,-0.207046,-0.106187,-0.229589,-0.209292,-0.162313,-0.192032,-0.217761,-0.228942,-0.136770,-0.160848,-0.183838,-0.135402,-0.166535,-0.109083,-0.103564,-0.139395,-0.221867
season,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
games_played,-0.061706,-0.277929,nan,1.000000,0.479367,0.710076,0.578391,0.668189,0.580307,0.618356,0.726633,0.629299,0.536749,0.610452,0.416487,0.579521,0.518525,0.469169,0.694331,0.659548,0.704899,0.536878,0.501136,0.678931,0.532662,0.486612,0.500439,0.474642,0.455846,0.718109
minutes/game,-0.015724,-0.186240,nan,0.479367,1.000000,0.556159,0.323263,0.495955,0.583720,0.694282,0.649417,0.582350,0.376974,0.439786,0.152670,0.363649,0.245277,0.109643,0.540233,0.719828,0.695836,0.222216,0.208857,0.561332,0.210120,0.252282,0.469846,0.267355,0.285552,0.583851
I_F_xOnGoal,0.032409,-0.216005,nan,0.710076,0.556159,1.000000,0.903884,0.984998,0.828330,0.738408,0.978418,0.910199,0.886962,0.884190,0.620145,0.431581,0.341808,0.180019,0.754784,0.687478,0.925015,0.839903,0.792151,0.977519,0.830849,0.804512,0.784467,0.781476,0.793324,0.998386
I_F_xGoals,0.049689,-0.168990,nan,0.578391,0.323263,0.903884,1.000000,0.948920,0.748050,0.589025,0.814021,0.856708,0.931006,0.856134,0.752928,0.300884,0.238693,0.116295,0.682473,0.480347,0.680186,0.970580,0.959595,0.848648,0.968883,0.967528,0.663870,0.873694,0.916354,0.883369
I_F_xRebounds,0.042071,-0.200261,nan,0.668189,0.495955,0.984998,0.948920,1.000000,0.823066,0.714603,0.941987,0.911818,0.910676,0.902446,0.678246,0.395821,0.313927,0.154721,0.740845,0.632562,0.859819,0.899305,0.859458,0.951360,0.892912,0.865584,0.755753,0.822297,0.837152,0.977737
I_F_primaryAssists,0.048165,-0.165493,nan,0.580307,0.583720,0.828330,0.748050,0.823066,1.000000,0.808730,0.827872,0.949532,0.782130,0.719357,0.502525,0.337983,0.245931,0.014117,0.750149,0.707934,0.777421,0.693208,0.625621,0.830294,0.680403,0.660919,0.725080,0.677506,0.682555,0.830575
I_F_secondaryAssists,0.049761,-0.181701,nan,0.618356,0.694282,0.738408,0.589025,0.714603,0.808730,1.000000,0.783176,0.860132,0.619788,0.625947,0.375540,0.350609,0.256973,0.082930,0.695234,0.744004,0.766908,0.513924,0.472432,0.742797,0.503056,0.502284,0.618938,0.518629,0.521169,0.751867


In [27]:
df_2022.drop(['_id', '', 'playerId','season','name','team','situation', 'position', 'Line_or_Pair', 'penalties', 'I_F_penalityMinutes', 'I_F_hits', 'I_F_primaryAssists', 'I_F_secondaryAssists', 'I_F_goals'],\
 axis='columns',inplace=True)

In [28]:
corr = df_2022.corr()
corr.style.background_gradient(cmap='coolwarm')

,games_played,minutes/game,I_F_xOnGoal,I_F_xGoals,I_F_xRebounds,I_F_shotAttempts,I_F_points,I_F_rebounds,I_F_reboundGoals,I_F_takeaways,I_F_giveaways,I_F_lowDangerShots,I_F_mediumDangerShots,I_F_highDangerShots,I_F_lowDangerxGoals,I_F_mediumDangerxGoals,I_F_highDangerxGoals,I_F_lowDangerGoals,I_F_mediumDangerGoals,I_F_highDangerGoals,I_F_unblockedShotAttempts
games_played,1.000000,0.479367,0.710076,0.578391,0.668189,0.726633,0.629299,0.610452,0.416487,0.694331,0.659548,0.704899,0.536878,0.501136,0.678931,0.532662,0.486612,0.500439,0.474642,0.455846,0.718109
minutes/game,0.479367,1.000000,0.556159,0.323263,0.495955,0.649417,0.582350,0.439786,0.152670,0.540233,0.719828,0.695836,0.222216,0.208857,0.561332,0.210120,0.252282,0.469846,0.267355,0.285552,0.583851
I_F_xOnGoal,0.710076,0.556159,1.000000,0.903884,0.984998,0.978418,0.910199,0.884190,0.620145,0.754784,0.687478,0.925015,0.839903,0.792151,0.977519,0.830849,0.804512,0.784467,0.781476,0.793324,0.998386
I_F_xGoals,0.578391,0.323263,0.903884,1.000000,0.948920,0.814021,0.856708,0.856134,0.752928,0.682473,0.480347,0.680186,0.970580,0.959595,0.848648,0.968883,0.967528,0.663870,0.873694,0.916354,0.883369
I_F_xRebounds,0.668189,0.495955,0.984998,0.948920,1.000000,0.941987,0.911818,0.902446,0.678246,0.740845,0.632562,0.859819,0.899305,0.859458,0.951360,0.892912,0.865584,0.755753,0.822297,0.837152,0.977737
I_F_shotAttempts,0.726633,0.649417,0.978418,0.814021,0.941987,1.000000,0.895329,0.845259,0.532876,0.754002,0.755997,0.971412,0.733764,0.681719,0.971063,0.721671,0.700368,0.795148,0.697818,0.709130,0.986755
I_F_points,0.629299,0.582350,0.910199,0.856708,0.911818,0.895329,1.000000,0.790467,0.607835,0.778150,0.712455,0.823711,0.799209,0.742315,0.899973,0.787920,0.775707,0.804796,0.805452,0.806123,0.908614
I_F_rebounds,0.610452,0.439786,0.884190,0.856134,0.902446,0.845259,0.790467,1.000000,0.633708,0.658597,0.560938,0.767126,0.808056,0.796119,0.837328,0.805959,0.791392,0.632897,0.703773,0.730733,0.877493
I_F_reboundGoals,0.416487,0.152670,0.620145,0.752928,0.678246,0.532876,0.607835,0.633708,1.000000,0.488198,0.276742,0.408546,0.733297,0.774231,0.559229,0.736092,0.763771,0.438826,0.665852,0.764762,0.598554
I_F_takeaways,0.694331,0.540233,0.754784,0.682473,0.740845,0.754002,0.778150,0.658597,0.488198,1.000000,0.670635,0.704631,0.627574,0.603463,0.745636,0.620586,0.609157,0.605334,0.594810,0.610068,0.756906


In [ ]:
df_2022.to_csv('model_2022-2023.csv')

# Preparing the Data

In [29]:
#Splitting the data
x = df_2022.copy()
x.drop('I_F_points', axis=1, inplace=True)
X = x
y = df_2022['I_F_points']

In [30]:
#further splitting into test and train data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [31]:
#creating the scaler variable
scaler = StandardScaler()

In [32]:
X_scaler = scaler.fit(X_train)

In [33]:
# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Linear Regression Model

In [34]:
linear_model = LinearRegression()

In [35]:
# Fit the data into the model
linear_model.fit(X_train_scaled, y_train)

LinearRegression()

In [36]:
# Display the model's best fit line formula
print(f"Model's formula: y = {linear_model.intercept_} + {linear_model.coef_[0]}X")

Model's formula: y = 28.33793103448276 + 0.6456004277572515X


In [37]:
#Make predictions with the model using X_test_scaled
predicted_y_values = linear_model.predict(X_test_scaled)


In [38]:
# Compute metrics for the linear regression model: score, r2, mse, rmse, std
score = linear_model.score(X_test_scaled, y_test, sample_weight=None)
r2 = r2_score(y_test, predicted_y_values)
mse = mean_squared_error(y_test, predicted_y_values)
rmse = np.sqrt(mse)
std = np.std(y_test)

# Print relevant metrics.
print(f"The score is {score}.")
print(f"The r2 is {r2}.")
print(f"The mean squared error is {mse}.")
print(f"The root mean squared error is {rmse}.")
print(f"The standard deviation is {std}.")

The score is 0.9349925224173266.
The r2 is 0.9349925224173266.
The mean squared error is 43.32669962894902.
The root mean squared error is 6.582302000740245.
The standard deviation is 25.81642765917697.


# Random Forest

In [51]:
# Create a random forest classifier
rf_model = RandomForestRegressor(n_estimators=1000, random_state=42)

In [52]:
# Fitting the model
rf_model_fit = rf_model.fit(X_train_scaled, y_train)

In [53]:
# Making predictions using the testing data
predictions = rf_model_fit.predict(X_test_scaled)

In [54]:
# Compute metrics for the linear regression model: score, r2, mse, rmse, std
score = rf_model_fit.score(X_test, y_test, sample_weight=None)
r2 = r2_score(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
std = np.std(y_test)

# Print relevant metrics.
print(f"The score is {score}.")
print(f"The r2 is {r2}.")
print(f"The mean squared error is {mse}.")
print(f"The root mean squared error is {rmse}.")
print(f"The standard deviation is {std}.")

The score is -7.755092286301911.
The r2 is 0.8994303912760117.
The mean squared error is 67.02843105154639.
The root mean squared error is 8.187089290556589.
The standard deviation is 25.81642765917697.


c:\Users\evan-\anaconda3\envs\PythonData\lib\site-packages\sklearn\base.py:444: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


# Neural Network

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=100, activation="relu", input_dim=number_input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=100, activation="relu"))

nn.add(tf.keras.layers.Dense(units=100, activation="relu"))

nn.add(tf.keras.layers.Dense(units=100, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="relu"))

# Check the structure of the model
nn.summary()

In [ ]:
# Compile the model
nn.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
# fit_model = nn.fit(X_train_scaled, y_train, epochs=100)
fit_model = nn.fit(X_train, y_train, epochs=100)

In [ ]:
# Evaluate the model using the test data
# model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [59]:
#saving the model
joblib.dump(linear_model, "my_linear_regression.joblib")


['my_linear_regression.joblib']